In [ ]:
import numpy as np
import pandas as pd
import math
import random
import sklearn
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CF

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CF


In [ ]:
import os
files = os.listdir('/content/drive/My Drive/CF/ml-100k/data')
data = {}

for file in files:
    data[file] = pd.read_csv('/content/drive/My Drive/CF/ml-100k/data/'+ file, sep='\t', encoding = 'latin-1', names = ['user_id', 'item_id', 'rating', 'timestamp'])
    data[file].drop(['timestamp'], axis = 1, inplace = True)

In [ ]:
# user-item ratings matrix
user_item_ratings = data['u.data'].pivot_table(index='user_id', columns='item_id', values='rating')
user_item_ratings = user_item_ratings.fillna(0)
user_item_ratings.index.name = None
user_item_ratings.columns.name = None

In [ ]:
# item-user ratings matrix
item_user_ratings = data['u.data'].pivot_table(index='item_id', columns='user_id', values='rating')
item_user_ratings = item_user_ratings.fillna(0)
item_user_ratings.index.name = None
item_user_ratings.columns.name = None
# item_user.reset_index(inplace = True, drop = True)

In [ ]:
# calculate cosine similarity
def calc_cosineSimilarity(x, y):
    dotprod = x.dot(y)
    x_mag = math.sqrt((x**2).sum())
    y_mag = math.sqrt((y**2).sum())
    sim = (float)(dotprod)/(x_mag*y_mag)
    return sim

In [ ]:
# generate similarities matrix
def calc_similarity(ratings, type, file_num):
    num_val = ratings.shape[0]
    similarities = pd.DataFrame(np.zeros((num_val, num_val)))
    for i in range(0, num_val, 1):
        for j in range(0, num_val, 1):
            x = ratings.iloc[i, :]
            y = ratings.iloc[j, :]
            similarities.iloc[i, j] = calc_cosineSimilarity(x, y)

    if type == 'user':
        if file_num == 0:
            similarities.to_csv('similarities.csv')
        else:
            file = 'similarities'+str(file_num)+'.csv'
            similarities.to_csv(file)
    else:
        if file_num == 0:
            similarities.to_csv('item_similarities.csv')
        else:
            file = 'item_similarities'+str(file_num)+'.csv'
            similarities.to_csv(file)
    return similarities

In [ ]:
# user-based similarities matrix
user_similarities = calc_similarity(user_item_ratings, 'user', 0)

In [ ]:
user_similarities = pd.read_csv('similarities.csv').iloc[:, 1:]
user_similarities.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942
count,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,...,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000
mean,0.245937,0.193984,0.124121,0.122596,0.173208,0.239909,0.243473,0.175839,0.103968,0.217116,0.194477,0.164810,0.268533,0.205479,0.185632,0.225169,0.156946,0.230806,0.110414,0.161584,0.164819,0.183028,0.218093,0.200764,0.191118,0.245040,0.118946,0.187390,0.132726,0.162248,0.079751,0.176888,0.128807,0.070315,0.098001,0.067402,0.158968,0.137779,0.139375,0.129885,...,0.117757,0.171590,0.139027,0.204914,0.190892,0.085110,0.225561,0.155262,0.131480,0.247248,0.048500,0.131221,0.242264,0.128231,0.108775,0.253662,0.141432,0.196132,0.201963,0.172326,0.208605,0.090030,0.164402,0.171970,0.121093,0.170443,0.163054,0.186979,0.181538,0.237543,0.195291,0.159771,0.185487,0.147434,0.212149,0.124575,0.230697,0.141349,0.171244,0.201141
std,0.140025,0.098894,0.105388,0.077707,0.128437,0.137484,0.160889,0.114940,0.052770,0.132561,0.100679,0.106995,0.123529,0.101284,0.097134,0.136398,0.097745,0.156423,0.061050,0.104353,0.077597,0.132123,0.132207,0.090923,0.109711,0.118013,0.077263,0.101203,0.080682,0.066282,0.064487,0.103827,0.126792,0.080276,0.107517,0.072810,0.119961,0.089007,0.124243,0.120378,...,0.071805,0.100641,0.094986,0.085773,0.116514,0.067121,0.092333,0.131490,0.116951,0.132656,0.057344,0.127317,0.155902,0.092661,0.096322,0.081361,0.137353,0.098735,0.129481,0.102640,0.096128,0.075202,0.134623,0.108755,0.061267,0.125985,0.075434,0.098402,0.147948,0.165051,0.125736,0.110162,0.095964,0.087787,0.120833,0.096052,0.084017,0.089256,0.084491,0.140712
min,0.002618,0.000000,0.000000,0.000000,0.000000,0.009023,0.012243,0.000000,0.000000,0.008601,0.004825,0.000000,0.062583,0.000000,0.000000,0.004042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.020939,0.000000,0.000000,0.000000,0.000000,0.000000,0.010366,0.000000,0.000000,0.000000,0.000000,0.000000,0.034229,0.000000,0.000000,0.000000,0.000000,0.018047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025313,0.000000,0.000000,0.000000
25%,0.135655,0.117829,0.051502,0.068067,0.068990,0.125008,0.104389,0.082884,0.073569,0.100894,0.114226,0.070058,0.165520,0.128676,0.120810,0.102580,0.088141,0.095538,0.072687,0.074436,0.115678,0.071999,0.110315,0.131730,0.107237,0.162422,0.064104,0.106299,0.077397,0.125780,0.034398,0.104319,0.037008,0.016112,0.020195,0.019612,0.059329,0.070226,0.050975,0.042498,...,0.069324,0.098042,0.073322,0.140598,0.088795,0.039512,0.161098,0.032379,0.021239,0.129553,0.006193,0.041583,0.119354,0.058094,0.026027,0.200428,0.045416,0.119403,0.095211,0.097931,0.138200,0.039694,0.065599,0.079643,0.080914,0.058899,0.107171,0.112084,0.057501,0.103295,0.085245,0.0

In [ ]:
# item-based similarities matrix
item_similarities = calc_similarity(item_user_ratings, 'item', 0)

In [ ]:
item_similarities = pd.read_csv('item_similarities.csv').iloc[:, 1:]
item_similarities.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,0.178794,0.168919,0.122236,0.197664,0.133840,0.065529,0.181805,0.178130,0.158491,0.114453,0.185040,0.189515,0.148091,0.123209,0.143218,0.076304,0.127719,0.067403,0.066931,0.098770,0.123491,0.190751,0.162498,0.151311,0.162209,0.129993,0.126753,0.192128,0.163848,0.105296,0.167866,0.126567,0.146810,0.049508,0.053544,0.090688,0.056614,0.162633,0.133980,0.122825,...,0.062305,0.061802,0.070810,0.048799,0.070810,0.070810,0.070810,0.070810,0.070810,0.014563,0.002969,0.004633,0.024084,0.019089,0.018285,0.031843,0.021552,0.021552,0.009249,0.032947,0.033691,0.043521,0.033691,0.033691,0.033691,0.033691,0.033691,0.033691,0.004756,0.019591,0.006351,0.012533,0.015867,0.015867,0.011147,0.009992,0.009992,0.009992,0.019815,0.018284
std,0.146738,0.138357,0.103266,0.149680,0.105049,0.068721,0.141491,0.135546,0.120332,0.090018,0.147094,0.145095,0.110762,0.104634,0.119789,0.068514,0.113940,0.080783,0.073085,0.086580,0.092075,0.153981,0.130471,0.121430,0.126346,0.097881,0.099933,0.155384,0.135985,0.093002,0.138946,0.103804,0.122134,0.069413,0.073803,0.076553,0.069997,0.131516,0.110956,0.098737,...,0.106659,0.129912,0.152593,0.100791,0.152593,0.152593,0.152593,0.152593,0.152593,0.054187,0.033013,0.033618,0.057681,0.058657,0.048986,0.075499,0.065788,0.065788,0.038164,0.120008,0.123388,0.083326,0.123388,0.123388,0.123388,0.123388,0.123388,0.123388,0.040460,0.051995,0.035629,0.046126,0.057483,0.057483,0.041519,0.059753,0.059753,0.059753,0.054532,0.050341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.059446,0.062125,0.035368,0.075218,0.050256,0.000454,0.064373,0.071605,0.064130,0.042603,0.065215,0.069048,0.061951,0.042632,0.047129,0.011957,0.035969,0.000000,0.007185,0.026855,0.050215,0.068086,0.065395,0.057371,0.061342,0.052640,0.050561,0.068366,0.056932,0.028612,0.062321,0.046097,0.047836,0.000000,0.000000,0.030973,0.000000,0.057520,0.039710,0.040083,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [ ]:
# data for cross validation
def ratings_matrix(data, num_users, num_items, type):
    num_val = data.shape[0]
    ratings = pd.DataFrame(np.zeros((num_users, num_items)))
    for i in range(num_val):
        user = data.iloc[i, 0]
        item = data.iloc[i, 1]
        rating = data.iloc[i, 2]
        ratings.iloc[user-1, item-1] = rating
    if type == 'item':
        ratings = ratings.transpose()
    return ratings

def prepare_ratings_data():
    user_ratings = []
    item_ratings = []
    test_data = [data['u1.test'], data['u2.test'], data['u3.test'], data['u4.test'], data['u5.test']]

    # user-item ratings matrix
    for i in range(5):
        file = 'u'+str(i+1)+'.base'
        user_item = ratings_matrix(data['u.data'], user_item_ratings.shape[0], user_item_ratings.shape[1], 'user')
        # user_item = data[file].pivot_table(index='user_id', columns='item_id', values='rating')
        user_item = user_item.fillna(0)
        user_ratings.append(user_item)

    # item-user ratings matrix
    for i in range(5):
        file = 'u'+str(i+1)+'.base'
        item_user = ratings_matrix(data['u.data'], user_item_ratings.shape[0], user_item_ratings.shape[1], 'item')
        # item_user = data['u1.base'].pivot_table(index='item_id', columns='user_id', values='rating')
        item_user = item_user.fillna(0)
        item_ratings.append(item_user)
    return user_ratings, item_ratings, test_data

def prepare_similarities_data():
    user_sim = []
    item_sim = []
    # user similarities matrix
    for i in range(5):
        file = 'similarities'+str(i+1)+'.csv'
        user_similarities = pd.read_csv(file).iloc[:, 1:]
        user_sim.append(user_similarities)

    # item similarities matrix
    for i in range(5):
        file = 'item_similarities'+str(i+1)+'.csv'
        item_similarities = pd.read_csv(file).iloc[:, 1:]
        item_sim.append(item_similarities)
    return user_sim, item_sim

In [ ]:
user_ratings, item_ratings, test_data = prepare_ratings_data()

In [ ]:
# user-based similarities matrix
for i in range(5):
    user_similarities = calc_similarity(user_ratings[i], 'user', i+1)

In [ ]:
# item-based similarities matrix
for i in range(5): 
    item_similarities = calc_similarity(item_ratings[i], 'item', i+1)

In [ ]:
user_sim, item_sim = prepare_similarities_data()

In [ ]:
# recommender system
def recommender_system(mode, ratings, similarities, curr_user, curr_item, topk, k):
    rating = 0
    sim_sum = 0
    if mode == 2:
        varmax = max(ratings.var(axis = 0))
        varmin = min(ratings.var(axis = 0))
        wt_sum = 0
    for neighbour in topk:
        neighbour_rating = ratings.iloc[neighbour, curr_item]
        neighbour_avg_rating = ratings.iloc[neighbour, :].mean()
        neighbour_sim = similarities.iloc[curr_user, neighbour]
        if neighbour_rating > 0:
            if mode == 0:
                sim_sum += neighbour_sim
                rating += ((neighbour_rating-neighbour_avg_rating)*neighbour_sim)
            elif mode == 1:
                sim_sum += neighbour_sim
                # significance weighting
                num_corated = (ratings.iloc[:, curr_item] != 0).sum()
                # threshold = 0.5*(ratings.iloc[curr_user, :] != 0).sum()
                threshold = k
                if num_corated < threshold:
                    rating += ((neighbour_rating-neighbour_avg_rating)*neighbour_sim*num_corated/threshold)
                else:
                    rating += ((neighbour_rating-neighbour_avg_rating)*neighbour_sim)
            else:
                # variance weighting
                vari = ratings.iloc[:, curr_item].var()
                weight = (vari - varmin)/(varmax)
                sim_sum += neighbour_sim*weight
                rating += ((neighbour_rating-neighbour_avg_rating)*neighbour_sim*weight)
    if sim_sum != 0:
        rating = (float) (rating) / (float)(sim_sum)
        rating += ratings.iloc[curr_user, :].mean()
    else:
        rating = 0
    return rating

In [ ]:
# user-based
def user_based(mode, ratings, similarities, user, item, k):
    sorted = list(user_similarities.sort_values(by = [str(user)], axis = 0, ascending = False).iloc[1:, :].index.values)
    topk = []
    ctr = 0
    for val in sorted:
        if ctr < k and ratings.iloc[val, item] > 0:
            topk.append(val)
            ctr += 1
    return recommender_system(mode, ratings, similarities, user, item, topk, k)

In [ ]:
# item-based
def item_based(mode, ratings, similarities, item, user, k):
    topk = item_similarities.sort_values(by = [str(item)], axis = 0, ascending = False).iloc[1:k+1, :].index.values
    return recommender_system(mode, ratings, similarities, item, user, topk, k)

In [ ]:
# five-fold cross validation
def cross_validation(type, mode, k, test_data):
    errors = []
    predicted_ratings = []
    for i in range(5):
        error = 0
        test_vals = test_data[i]
        for j in range(test_vals.shape[0]):
            user = test_vals.iloc[j, 0]-1
            item = test_vals.iloc[j, 1]-1
            rating = test_vals.iloc[j, 2]
            if type == 'user':
                pred_rating = user_based(mode, user_ratings[i], user_sim[i], user, item, k)
            else:
                pred_rating = item_based(mode, item_ratings[i], item_sim[i], item, user, k)
            predicted_ratings.append(pred_rating)
            error += abs(pred_rating - rating)
        errors.append(error/test_vals.shape[0])
    return sum(errors)/5.0, errors, predicted_ratings

# Results

In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 10
mode = 0
user_score, user_scores, predicted_ratings = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score, user_scores)

0.817159059697213 [0.8274772411798406, 0.8172548038516511, 0.8165493693634971, 0.8117027814599426, 0.8128111026311341]


In [ ]:
k = 10
mode = 0
item_score, item_scores, predicted_ratings = cross_validation('item', mode, k, test_data)

In [ ]:
print(item_score, item_scores)

0.8170230985256884 [0.815309504055477, 0.8076869148351671, 0.8205200145565448, 0.8203239160659003, 0.8212751431153525]


In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 20
mode = 0
user_score_2, user_scores_2, predicted_ratings_2 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_2, user_scores_2)

0.8107233157883902 [0.821782862040566, 0.8133610377891127, 0.8100931360950503, 0.803341561385033, 0.8050379816321894]


In [ ]:
k = 20
mode = 0
item_score_2, item_scores_2, item_predicted_ratings_2 = cross_validation('item', mode, k, test_data)

In [ ]:
print(item_score_2, item_scores_2)

0.7664408335035224 [0.7655768664522675, 0.7599337797592642, 0.76981944318276, 0.7673388491799868, 0.7695352289433333]


In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 30
mode = 0
user_score_3, user_scores_3, predicted_ratings_3 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_3, user_scores_3)

0.810871335102059 [0.8213998659991896, 0.813673874602783, 0.8100849344041721, 0.8032922356643806, 0.8059057648397704]


In [ ]:
k = 30
mode = 0
item_score_3, item_scores_3, item_predicted_ratings_3 = cross_validation('item', mode, k, test_data)

In [ ]:
print(item_score_3, item_scores_3)

0.7558445233429343 [0.7587153269879793, 0.7490119621162711, 0.7574375914599948, 0.7565151877357612, 0.7575425484146646]


In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 40
mode = 0
user_score_4, user_scores_4, predicted_ratings_4 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_4, user_scores_4)

0.8125335107771067 [0.8231843024182417, 0.8158467092323171, 0.8109795199090807, 0.8046652797008899, 0.8079917426250036]


In [ ]:
k = 40
mode = 0
item_score_4, item_scores_4, item_predicted_ratings_4 = cross_validation('item', mode, k, test_data)

In [ ]:
print(item_score_4, item_scores_4)

0.7517686246227233 [0.7559341434285592, 0.7450563468428884, 0.7541808491902556, 0.751117760303084, 0.7525540233488293]


In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 50
mode = 0
user_score_5, user_scores_5, predicted_ratings_5 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_5, user_scores_5)

0.8142792968016019 [0.8248198130913128, 0.8176261704697229, 0.81287341613864, 0.8060259610975133, 0.8100511232108206]


In [ ]:
k = 50
mode = 0
item_score_5, item_scores_5, item_predicted_ratings_5 = cross_validation('item', mode, k, test_data)

In [ ]:
print(item_score_5, item_scores_5)

0.7506848280933347 [0.7553385736682008, 0.7441692577705815, 0.7530533236907719, 0.7494332782934705, 0.7514297070436494]


# Significance Weighting

In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 10
mode = 1
user_score_sw, user_scores_sw, predicted_ratings_sw = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_sw, user_scores_sw)

0.8201438371925118 [0.8276077250163034, 0.8201976484261743, 0.8203597438790518, 0.8155018390449208, 0.8170522295961082]


In [ ]:
k = 20
mode = 1
user_score_sw_2, user_scores_sw_2, predicted_ratings_sw_2 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_sw_2, user_scores_sw_2)

0.8257298933141085 [0.8321248722486422, 0.8273139732328233, 0.826501290469367, 0.8207682720420975, 0.8219410585776122]


In [ ]:
k = 30
mode = 1
user_score_sw_3, user_scores_sw_3, predicted_ratings_sw_3 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_sw_3, user_scores_sw_3)

0.8407601962714585 [0.8453795277870769, 0.842447153797756, 0.8405811732128637, 0.8366707796965825, 0.8387223468630134]


# Variance Weighting

In [ ]:
# result evaluation
# regular: mode = 0, significance weighting: mode = 1, variance weighting: mode = 2
k = 10
mode = 2
user_score_vw, user_scores_vw, predicted_ratings_vw = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_vw, user_scores_vw)

0.817159059697213 [0.8274772411798406, 0.8172548038516513, 0.8165493693634971, 0.8117027814599426, 0.8128111026311339]


In [ ]:
k = 20
mode = 2
user_score_vw_2, user_scores_vw_2, predicted_ratings_vw_2 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_vw_2, user_scores_vw_2)

0.8107233157883904 [0.8217828620405666, 0.8133610377891127, 0.8100931360950503, 0.8033415613850329, 0.8050379816321894]


In [ ]:
k = 30
mode = 2
user_score_vw_3, user_scores_vw_3, predicted_ratings_vw_3 = cross_validation('user', mode, k, test_data)

In [ ]:
print(user_score_vw_3, user_scores_vw_3)

0.8108713351020592 [0.8213998659991898, 0.8136738746027833, 0.8100849344041724, 0.8032922356643803, 0.8059057648397701]
